# Discrete Model.

In [1]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from pyomo.environ import * 
import pandas
import math

In [6]:
%%writefile oil_export.dat

param n := 11;

Spain  France, Italy, Switzerland, Belgium, Monaco, Netherland, Germany, Poland, UK,  Ireland

Ireland Poland Italy Switzerland France Monaco UK Belgium Netherlands Germany;


param Cost : 1  2  3  4  5  6  7  8  9  10  11
1   0  10  10  10  10  10  10  10  10 10   10
2   10  0  60 4  302  1  10  10  30  40   15
3   10  40  0  20  30  40  20  20  50  20   20
4   10  20  30  0  60  60  70  30  20  10   30
5   10  50  40  50  0  20  70  40  50  50   30
6   10  30  60  30  50  0  70  20  10  20   20
7   10  70  40  30  40  20  0  60  70  80   30
8   10  20  1  20  20  30  60  70  0  90   30
9   10  10  9  50  40  20  20  10  0  10   50
10  10  20  2  30  20  40  10  20  30  0   60
11  10  30  5  40  40  80  70  50  50  50   0
;





Writing oil_export.dat


In [87]:
from pyomo.environ import *

# Create Pyomo model
model = ConcreteModel()

# Sets
model.Warehouses = RangeSet(1, 5)  # Example: 5 potential warehouse locations
model.Customers = RangeSet(1, 7)  # Example: 4 customers


# Madrid, barcelona, Malaga, Coruna, Sevilla
# 
cost = {
    (1,1): 10,
    (1,2): 4,
    (1,3): 5,
    (1,4): 7,
    (1,5): 4,
    (1,6): 8,
    (1,7): 10,

    (2,1): 15,
    (2,2): 2,
    (2,3): 6,
    (2,4): 8,
    (2,5): 2,
    (2,6): 2,
    (2,7): 1,

    (3,1): 1,
    (3,2): 9,
    (3,3): 2,
    (3,4): 5,
    (3,5): 2,
    (3,6): 9,
    (3,7): 9,

    (4,1): 3,
    (4,2): 2,
    (4,3): 1,
    (4,4): 2,
    (4,5): 5,
    (4,6): 3,
    (4,7): 1,

    (5,1): 2,
    (5,2): 1,
    (5,3): 2,
    (5,4): 5,
    (5,5): 3,
    (5,6): 2,
    (5,7): 8
}

# Parameters
model.Cost = Param(model.Warehouses, model.Customers, initialize=cost)

model.Stock = Param(model.Warehouses, initialize = {1: 500, 2: 500, 3: 600, 4: 400, 5:330})

# Demand for each customer
model.Demand = Param(model.Customers, initialize={1: 50, 2: 30, 3: 40, 4: 20, 5:80, 6:90, 7:20})

# Decision variables
model.y = Var(model.Warehouses, within=Binary)


# Objective function
model.obj = Objective(expr=sum(model.Cost[i, j] * model.y[i] * model.Demand[j] for i in model.Warehouses for j in model.Customers), sense=minimize)

# Constraints
model.demand_fulfillment = ConstraintList()
for j in model.Customers:
    model.demand_fulfillment.add(sum(model.y[i] * model.Demand[j] for i in model.Warehouses) >= model.Demand[j])

# model.warehouse_sol = ConstraintList()
# for i in model.Warehouses:
#     model.warehouse_sol.add(sum(model.y[i])>=2)


def two_warehouses(model, i):
    return sum(model.y[i] for i in model.Warehouses ) == 2

model.warehouses_two = Constraint(model.Warehouses, rule = two_warehouses)


# def ensure_stock_rule(model, i):
#     return model.Stock[i] >= sum(model.Demand[j] for j in model.Customers)

# model.warehouses_stock = Constraint(model.Warehouses, rule=ensure_stock_rule)

def ensure_stock_rule(model, i):
    return sum(model.y[i]*model.Demand[j] for j in model.Customers) <= model.Stock[i]

model.warehouses_stock = Constraint(model.Warehouses, rule=ensure_stock_rule)



# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)


display(model)

Model unknown

  Variables:
    y : Size=5, Index=Warehouses
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1870.0

  Constraints:
    demand_fulfillment : Size=7
        Key : Lower : Body  : Upper
          1 :  50.0 : 100.0 :  None
          2 :  30.0 :  60.0 :  None
          3 :  40.0 :  80.0 :  None
          4 :  20.0 :  40.0 :  None
          5 :  80.0 : 160.0 :  None
          6 :  90.0 : 180.0 :  None
          7 :  20.0 :  40.0 :  None
    warehouses_two : Size=5
        Key : Lower : Body : Upper
          1 :   2.0 :  2.0 :   2.0
          2 :   2.0 :

In [42]:
from pyomo.environ import *

# Create Pyomo model
model = ConcreteModel()

# Sets
model.Warehouses = RangeSet(1, 5)  # Example: 5 potential warehouse locations
model.Customers = RangeSet(1, 4)  # Example: 4 customers

# Parameters
model.Cost = Param(model.Warehouses, model.Customers, initialize={(i, j): i + j for i in model.Warehouses for j in model.Customers})
model.CustomerDemand = Param(model.Customers, initialize={1: 10, 2: 15, 3: 8, 4: 12})
model.WarehouseDemand = Param(model.Warehouses, initialize={1: 5, 2: 8, 3: 12, 4: 15, 5: 7})
model.WarehouseSupply = Param(model.Warehouses, initialize={1: 20, 2: 25, 3: 15, 4: 18, 5: 10})

# Decision variables
model.y = Var(model.Warehouses, within=Binary)

# Objective function
model.obj = Objective(expr=sum(model.Cost[i, j] * model.y[i] * (model.CustomerDemand[j] + model.WarehouseDemand[i] - model.WarehouseSupply[i]) for i in model.Warehouses for j in model.Customers), sense=minimize)

# Constraints
model.customer_demand_fulfillment = ConstraintList()
for j in model.Customers:
    model.customer_demand_fulfillment.add(sum(model.y[i] * model.CustomerDemand[j] for i in model.Warehouses) >= model.CustomerDemand[j])

model.warehouse_demand_fulfillment = ConstraintList()
for i in model.Warehouses:
    model.warehouse_demand_fulfillment.add(sum(model.Cost[i, j] * model.y[i] * (model.CustomerDemand[j] + model.WarehouseDemand[i] - model.WarehouseSupply[i]) for j in model.Customers) >= model.WarehouseDemand[i])

model.warehouse_supply_constraint = ConstraintList()
for i in model.Warehouses:
    model.warehouse_supply_constraint.add(sum(model.Cost[i, j] * model.y[i] * (model.CustomerDemand[j] + model.WarehouseDemand[i] - model.WarehouseSupply[i]) for j in model.Customers) <= model.WarehouseSupply[i])

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Display results
display(model)


Model unknown

  Variables:
    y : Size=5, Index=Warehouses
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: y[1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object y[1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object y[1]
        Key : Active : Value
        None :   None :  None

  Constraints:
    customer_demand_fulfillment : Size=4
        Key : Lower : Body : Upper
          1 :  10.0 : No